In [4]:
import tensorflow as tf
from __future__ import division, print_function, absolute_import
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [28]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 784
num_class = 10
dropout =0.75

def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['images']
        
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(conv2)
        
        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, n_classes)
        
    return out

In [29]:
def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_class, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_class, dropout, reuse=True, is_training=False)
    
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        train_op=train_op,
        loss=loss_op,
        eval_metric_ops={'accuracy': acc_op})
    
    return estim_specs
        

In [30]:
model = tf.estimator.Estimator(model_fn)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, shuffle=False)
model.train(input_fn, steps=num_steps)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
model.train(input_fn, steps=num_steps)
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Wayne-PC\\AppData\\Local\\Temp\\tmp_2h4u8e1', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Wayne-PC\AppData\Local\Temp\tmp_2h4u8e1\model.ckpt.
INFO:tensorflow:loss = 2.38635, step = 1
INFO:tensorflow:global_step/sec: 207.827
INFO:tensorflow:loss = 0.278482, step = 101 (0.482 sec)
INFO:tensorflow:global_step/sec: 214.698
INFO:tensorflow:loss = 0.132922, step = 201 (0.466 sec)
INFO:tensorflow:global_step/sec: 211.757
INFO:tensorflow:loss = 0.108765, step = 301 (0.472 sec)
INFO:tensorflow:global_step/sec: 214.055
INFO:tensorflow:loss = 0.0917427, step = 401 (0.467 sec)
INFO:tensorflow:Saving checkpoints for 43